<a href="https://colab.research.google.com/github/Zeropeepo/project-DATATHON/blob/angeloBranch/data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.model_selection import train_test_split
import seaborn as sns
from scipy import stats

# **Should This Loan Be Approved Or Not Dataset**

The dataset is from the U.S. Small Business Administration (SBA). Contains 899164 rows and 27 columns

Columns:
1. **LoanNr_ChkDgt** = Identifier Primary key
2. **Name** = Borrower name
3. **City** = Borrower city
4. **State** = Borrower state
5. **Zip** = Borrower zip code
6. **Bank** = Bank name
7. **BankState** = Bank state
8. **NAICS** = North American industry classification system code
9. **ApprovalDate** = Date SBA commitment issued
10. **ApprovalFY** = Fiscal year of commitment
11. **Term** = Loan term in months
12. **NoEmp** = Number of business employees
13. **NewExist** = 1: Existing business, 2: New business
14. **CreateJob** = Number of jobs created
15. **RetainedJob** = Number of jobs retained
16. **FranchiseCode** = Franchise code, 0 or 1: No franchise
17. **UrbanRural** = 1: Urban, 2: rural, 0: undefined
18. **RevLineCr** = Revolving line of credit, Y: Yes, N: No
19. **LowDoc** = LowDoc Loan Program, Y: Yes, N: No
20. **ChgOffDate** = The date when a loan is declared to be in default
21. **DisbursementDate** = Disbursement date
22. **DisbursementGross** = Amount disbursed
23. **BalanceGross** = Gross amount outstanding
24. **MIS_Status** = Loan status charged off = CHGOFF, Paid in full =PIF
25. **ChgOffPrinGr** = Charged-off amount
26. **GrAppv** = Gross amount of loan approved by bank
27. **SBA_Appv** = SBA's guaranteed amount of approved loan



## Load Dataset From Huggingface

In [3]:
!pip install -U datasets

In [7]:
from datasets import load_dataset, Features, Value

# 1. Download URL for CSV file
csv_url = "https://huggingface.co/datasets/alflax77/XAI_KreditLensa/resolve/main/KreditLensa_Dataset.csv"
dataset_id = "alflax77/Dataset_KreditLensa"
file_name = "KreditLensa_Dataset.csv"

# 2. Read only the header (nrows=0) to get column names
try:
    df_header = pd.read_csv(csv_url, nrows=0)
    column_names = df_header.columns.tolist()

    # 3. Automatically create the feature schema with all columns as strings
    feature_schema = Features({col: Value('string') for col in column_names})

    # 4. Load the dataset using the generated schema
    dataset = load_dataset(
        dataset_id,
        data_files=file_name,
        features=feature_schema,
        split='train'
    )

    print("✅ Dataset loaded successfully!")

except Exception as e:
    print(f"An error occurred: {e}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/33.0 [00:00<?, ?B/s]

KreditLensa_Dataset.csv:   0%|          | 0.00/179M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

✅ Dataset loaded successfully!


In [ ]:
# changing Dataset Object to Pandas Dataframe
df = dataset.to_pandas()

In [ ]:
df.head()

,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,...,RevLineCr,LowDoc,ChgOffDate,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv
0,1000014003,ABC HOBBYCRAFT,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,451120,28-Feb-97,1997,...,N,Y,None,28-Feb-99,"$60,000.00",$0.00,P I F,$0.00,"$60,000.00","$48,000.00"
1,1000024006,LANDMARK BAR & GRILLE (THE),NEW PARIS,IN,46526,1ST SOURCE BANK,IN,722410,28-Feb-97,1997,...,N,Y,None,31-May-97,"$40,000.00",$0.00,P I F,$0.00,"$40,000.00","$32,000.00"
2,1000034009,"WHITLOCK DDS, TODD M.",BLOOMINGTON,IN,47401,GRANT COUNTY STATE BANK,IN,621210,28-Feb-97,1997,...,N,N,None,31-Dec-97,"$287,000.00",$0.00,P I F,$0.00,"$287,000.00","$215,250.00"
3,1000044001,"BIG BUCKS PAWN & JEWELRY, LLC",BROKEN ARROW,OK,74012,1ST NATL BK & TR CO OF BROKEN,OK,0,28-Feb-97,1997,...,N,Y,None,30-Jun-97,"$35,000.00",$0.00,P I F,$0.00,"$35,000.00","$28,000.00"
4,1000054004,"ANASTASIA CONFECTIONS, INC.",ORLANDO,FL,32801,FLORIDA BUS. DEVEL CORP,FL,0,28-Feb-97,1997,...,N,N,None,14-May-97,"$229,000.00",$0.00,P I F,$0.00,"$229,000.00","$229,000.00"


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 899164 entries, 0 to 899163
Data columns (total 27 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   LoanNr_ChkDgt      899164 non-null  object
 1   Name               899150 non-null  object
 2   City               899134 non-null  object
 3   State              899150 non-null  object
 4   Zip                899164 non-null  object
 5   Bank               897605 non-null  object
 6   BankState          897598 non-null  object
 7   NAICS              899164 non-null  object
 8   ApprovalDate       899164 non-null  object
 9   ApprovalFY         899164 non-null  object
 10  Term               899164 non-null  object
 11  NoEmp              899164 non-null  object
 12  NewExist           899028 non-null  object
 13  CreateJob          899164 non-null  object
 14  RetainedJob        899164 non-null  object
 15  FranchiseCode      899164 non-null  object
 16  UrbanRural         8

## Data Cleaning

In [ ]:
df1 = df.copy()

In [ ]:
# drop features that can't be implemented in Indonesia
df1.drop(['LoanNr_ChkDgt', 'Name', 'City', 'State', 'Zip', 'Bank', 'BankState', 'ApprovalDate', 'ApprovalFY', 'LowDoc', 'ChgOffDate', 'BalanceGross', 'ChgOffPrinGr', 'DisbursementDate', 'SBA_Appv'], axis=1, inplace=True)

In [ ]:
# drop GrAppv because the function is more or less the same as Disbursement gross
# disbursement gross = the amount of loan a customer take
# grappv (gross approval) = the maximum amount of loan that bank can give
df1.drop('GrAppv', axis=1, inplace=True)

In [ ]:
# change the data types of 'Term', 'NoEmp', 'CreateJob', 'RetainedJob' to int64
df1['Term'] = df1['Term'].astype('int64')
df1['NoEmp'] = df1['NoEmp'].astype('int64')
df1['CreateJob'] = df1['CreateJob'].astype('int64')
df1['RetainedJob'] = df1['RetainedJob'].astype('int64')

In [ ]:
# only take the first 2 digits of NAICS
df1['NAICS'] = df1['NAICS'].apply(lambda x: int(str(x)[:2]))

In [ ]:
naics_dict = {
    11: "Agriculture, Forestry, Fishing and Hunting",
    21: "Mining, quarrying, and oil and gas extraction",
    22: "Utilities",
    23: "Construction",
    31: "Manufacturing",
    32: "Manufacturing",
    33: "Manufacturing",
    42: "Wholesale trade",
    44: "Retail trade",
    45: "Retail trade",
    48: "Transportation and warehousing",
    49: "Transportation and warehousing",
    51: "Information",
    52: "Finance and insurance",
    53: "Real estate and rental and leasing",
    54: "Professional, scientific, and technical services",
    55: "Management of companies and enterprises",
    56: "Administrative and support and waste management and remediation services",
    61: "Educational services",
    62: "Health care and social assistance",
    71: "Arts, entertainment, and recreation",
    72: "Accommodation and food services",
    81: "Other services (except public administration)",
    92: "Public administration",
    0: "Not Stated"
}

In [ ]:
# change NAICS digits into the real value of industry job classification
df1['NAICS'] = df1['NAICS'].map(naics_dict)

In [ ]:
#change unit from dollar to rupiah
def dollar_to_rp(x):
    dollar = float(x.replace(',', '').replace('$', ''))
    rupiah = dollar*16500
    rupiah_normalized = int(rupiah/15.5)     # dibagi dengan skala biaya hidup di US vs Indo
    return float(rupiah_normalized)

df1['DisbursementGross'] = df1['DisbursementGross'].apply(dollar_to_rp)

In [ ]:
# change FranchiseCode value into object:
# 0 or 1 --> no franchise (0)
# others --> has franchise (1)
df1['FranchiseCode'] = df1['FranchiseCode'].apply(lambda x: '0' if x in [0, 1] else '1')

In [ ]:
# change 'NewExist', 'FranchiseCode', 'UrbanRural' into object/categorical data
df1['NewExist'] = df1['NewExist'].apply(lambda x: '1' if x == 1 else '2')
df1['UrbanRural'] = df1['UrbanRural'].astype('object')

In [ ]:
# drop the rest of the NaN value
df1.dropna(inplace=True)

In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 892647 entries, 0 to 899163
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   NAICS              892647 non-null  object 
 1   Term               892647 non-null  int64  
 2   NoEmp              892647 non-null  int64  
 3   NewExist           892647 non-null  object 
 4   CreateJob          892647 non-null  int64  
 5   RetainedJob        892647 non-null  int64  
 6   FranchiseCode      892647 non-null  object 
 7   UrbanRural         892647 non-null  object 
 8   RevLineCr          892647 non-null  object 
 9   DisbursementGross  892647 non-null  float64
 10  MIS_Status         892647 non-null  object 
dtypes: float64(1), int64(4), object(6)
memory usage: 81.7+ MB


## Simple modelling with catboost

In [ ]:
pip install catboost

In [ ]:
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
categorical_data = []
numerical_data = []
target = 'MIS_Status'
for i in df1.drop(target, axis=1).columns:
    if df1[i].dtype == 'object':
        categorical_data.append(i)
    else:
        numerical_data.append(i)

**Total 10 fitur/kolom**

In [ ]:
X = df1.drop(target, axis=1)
y = df1[target]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# Create a CatBoost Pool
train_pool = Pool(data=X_train, label=y_train, cat_features=categorical_data)
test_pool = Pool(data=X_test, label=y_test, cat_features=categorical_data)

# Initialize the CatBoostClassifier
model = CatBoostClassifier(
    iterations=100,
    learning_rate=0.1,
    depth=3,
    loss_function='Logloss',
    verbose=False  # Suppress output during training
)

# Train the model
model.fit(train_pool)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}\n')
print(classification_report(y_test, y_pred))

Accuracy: 0.92

              precision    recall  f1-score   support

      CHGOFF       0.81      0.69      0.75     46853
       P I F       0.94      0.97      0.95    220942

    accuracy                           0.92    267795
   macro avg       0.87      0.83      0.85    267795
weighted avg       0.91      0.92      0.92    267795

